# Setup

In [1]:
%%capture
%%bash
python ./generate_data.py
python ./run_ddl.py

Run Python code as shown below

In [2]:
a = 10

Run SQL code as shown below, with the `%%sql` called magics

In [3]:
%%sql
select 1

,1
0,1


We use the `prod.db` schema where all our tables are create by `run_ddl.py`

In [4]:
%%sql 
use prod_db

""


## Your code below

In [5]:
%%sql 
WITH orders_cte AS (
    SELECT
        o_orderkey,
        o_custkey,
        o_orderstatus,
        CAST(o_orderdate AS TIMESTAMP) AS o_orderdate,
        o_orderpriority,
        o_clerk,
        o_shippriority,
        o_comment,
        o_totalprice
    FROM orders
),
stg_customers AS (
    SELECT
        c_custkey,
        c_name,
        c_address,
        c_nationkey,
        c_phone,
        c_acctbal,
        c_mktsegment,
        c_comment
    FROM customer
),
nation_cte AS (
    SELECT
        CAST(n_nationkey AS INT) AS n_nationkey,
        CAST(n_name AS STRING) AS n_name,
        CAST(n_regionkey AS INT) AS n_regionkey,
        CAST(n_comment AS STRING) AS n_comment
    FROM nation
),
dim_customers AS (
    SELECT
        c.c_custkey,
        c.c_name,
        c.c_address,
        c.c_nationkey,
        n.n_name AS nation_name,
        c.c_phone,
        c.c_acctbal,
        c.c_mktsegment,
        c.c_comment
    FROM stg_customers c
    INNER JOIN nation_cte n ON c.c_nationkey = n.n_nationkey
)
SELECT
    o.o_orderkey,
    o.o_custkey,
    o.o_orderstatus,
    o.o_orderdate,
    o.o_orderpriority,
    o.o_clerk,
    o.o_shippriority,
    o.o_totalprice,
    c.c_name AS customer_name,
    c.c_address AS customer_address,
    c.c_phone AS customer_phone,
    c.c_acctbal AS customer_account_balance,
    c.c_mktsegment AS customer_market_segment,
    c.nation_name AS customer_nation_name
FROM orders_cte o
INNER JOIN dim_customers c ON o.o_custkey = c.c_custkey;

,o_orderkey,o_custkey,o_orderstatus,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_totalprice,customer_name,customer_address,customer_phone,customer_account_balance,customer_market_segment,customer_nation_name
0,897,4894,P,1995-03-20,1-URGENT,Clerk#000000316,0,66775.72,Customer#000004894,qccDi3BeqTSkIJKsp21j4KBsQpPk0Gt,30-152-685-8820,7927.44,HOUSEHOLD,SAUDI ARABIA
1,4034,9233,F,1993-11-14,4-NOT SPECIFIED,Clerk#000000548,0,307637.71,Customer#000009233,1Ba3hAO5jmBFJpKq2QIdZlb3Z,33-327-912-9200,4227.72,FURNITURE,UNITED KINGDOM
2,4769,12044,P,1995-04-14,4-NOT SPECIFIED,Clerk#000000116,0,206126.35,Customer#000012044,u9bevlA h3PS,10-488-427-2414,4825.26,MACHINERY,ALGERIA
3,5123,964,O,1998-02-10,1-URGENT,Clerk#000000776,0,18908.75,Customer#000000964,"2DpdvfAw4TuXl8DV,YVBwGsK7oLzkYr",22-974-772-2802,4756.58,FURNITURE,JAPAN
4,6695,1697,F,1992-06-28,5-LOW,Clerk#000000877,0,322140.76,Customer#000001697,8aHZSJIoZKtIfcD8IFMCu,34-288-313-5272,-913.24,FURNITURE,UNITED STATES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,370917,13240,O,1996-11-20,4-NOT SPECIFIED,Clerk#000000170,0,120765.87,Customer#000013240,"PJ,znldxQIQzPz1OUJ,A2edR37Sv9gpL1eJ3k",34-177-589-8705,5764.35,FURNITURE,UNITED STATES
996,370980,6424,O,1996-09-16,1-URGENT,Clerk#000000964,0,138463.90,Customer#000006424,JM18rGuDap8AhcSqcLsTFObP6 DF8,16-525-665-8732,258.29,MACHINERY,FRANCE
997,371043,6424,O,1995-04-30,5-LOW,Clerk#000000477,0,37138.82,Customer#000006424,JM18rGuDap8AhcSqcLsTFObP6 DF8,16-525-665-8732,258.29,MACHINERY,FRANCE
998,371559,5759,F,1992-03-13,4-NOT SPECIFIED,Clerk#000000609,0,54661.17,Customer#000005759,KH OTbvxFKOB40,24-556-307-7039,313.45,HOUSEHOLD,KENYA


In [6]:
# In Jupyter
for conf in spark.sparkContext.getConf().getAll():
    if 'iceberg' in conf[0].lower() or 'catalog' in conf[0].lower():
        print(f"{conf[0]}: {conf[1]}")

spark.sql.catalog.local: org.apache.iceberg.spark.SparkCatalog
spark.sql.catalog.local.warehouse: file:///home/airflow/warehouse
spark.sql.defaultCatalog: local
spark.sql.catalog.local.type: hadoop
spark.sql.catalogImplementation: in-memory


In [7]:
spark.version

'4.1.1'

In [8]:
%%sql
select 1

,1
0,1


In [9]:
# Check the actual JAR being used
import glob
jars = spark.sparkContext._jsc.sc().listJars()
for jar in jars:
    if 'iceberg' in jar.toString().lower():
        print(jar)

# Also check
print(spark.conf.get("spark.jars"))

TypeError: 'JavaObject' object is not iterable

In [10]:
spark.version
spark.conf.get("spark.jars")
# Or check what's actually loaded:
spark.sparkContext._conf.getAll()

[('spark.hadoop.fs.s3a.vectored.read.min.seek.size', '128K'),
 ('spark.repl.local.jars',
  'file:///opt/spark/jars/iceberg-spark-runtime-4.0_2.13-1.10.1.jar'),
 ('spark.sql.artifact.isolation.enabled', 'false'),
 ('spark.executor.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-modules=jdk.incubator.vector --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=

In [13]:
! pwd

/home/airflow/notebooks


In [14]:
! uv run dbt debug --profiles-dir /home/airflow/tpch_analytics/ --project-dir /home/airflow/tpch_analytics/

19:59:34  Running with dbt=1.11.5
19:59:34  dbt version: 1.11.5
19:59:34  python version: 3.13.12
19:59:34  python path: /home/airflow/.venv/bin/python3
19:59:34  os info: Linux-6.17.9-76061709-generic-x86_64-with-glibc2.36
19:59:34  Using profiles dir at /home/airflow/tpch_analytics/
19:59:34  Using profiles.yml file at /home/airflow/tpch_analytics/profiles.yml
19:59:34  Using dbt_project.yml file at /home/airflow/tpch_analytics/dbt_project.yml
19:59:34  adapter type: spark
19:59:34  adapter version: 1.10.1
19:59:35  Configuration:
19:59:35    profiles.yml file [OK found and valid]
19:59:35    dbt_project.yml file [OK found and valid]
19:59:35  Required dependencies:
19:59:35   - git [OK found]

19:59:35  Connection:
19:59:35    host: localhost
19:59:35    port: 443
19:59:35    cluster: None
19:59:35    endpoint: None
19:59:35    schema: prod_db
19:59:35    organization: 0
19:59:35  Registered adapter: spark=1.10.1
26/02/13 19:59:37 WARN NativeCodeLoader: Unable to load native-hadoop 

In [17]:
spark.sql(" SELECT 1 as id").show()

+---+
| id|
+---+
|  1|
+---+

